#Imports

In [ ]:
from google.colab import drive

import numpy as np
import os
import pandas as pd

#Mount Drive

In [ ]:
drive.mount('/content/drive')

#Constants

In [ ]:
dir = "/content/drive/MyDrive/Thesis/"

In [ ]:
sources = ['CHIRPS', 'MODIS', 'S1', 'S2']

In [ ]:
NUM_POINTS_PER_SEED = 5000

#Dataframes Preparation, Cleaning

In [ ]:
points_data = []
path = dir + 'MODIS'
# seeds = sorted([f.split("__seed_")[1][:-4] for f in os.listdir(path) if f.endswith('.csv')])
seeds = sorted(list(set([f.split("__seed_")[1][:-4] for f in os.listdir(path) if f.endswith('0.csv')])))
print(seeds)
for seed in seeds:
  # points_col = pd.read_csv(path + '/lst__seed_' + seed + '.csv')['.geo']
  points_col = pd.read_csv(path + '/lst_k__seed_' + seed + '.csv')['.geo']
  for i in range(NUM_POINTS_PER_SEED):
    str_list = points_col[i][31:-2].split(',')
    # print(str_list)
    float_list = [float(c) for c in str_list]
    # print(float_list)
    points_data.append(float_list)

In [ ]:
# Create the pandas DataFrame
points_df = pd.DataFrame(points_data, columns = ['lon', 'lat'])

In [ ]:
# print dataframe.
points_df

In [ ]:
path = dir + 'output/'
points_df.to_csv(path + 'points.csv')

*** MODIS Only

In [ ]:
lst_files = ['lst_c', 'lst_k']
l_seeds = ['seed_120', 'seed_30', 'seed_60', 'seed_90']
for f in lst_files:
  print(f)
  dir_lst = {}
  for s in l_seeds:
    dir_lst[s] = {}
    dir_lst[s]['df'] = pd.read_csv(dir + 'MODIS/' + f + '__' + s + '.csv')
    dir_lst[s]['cols'] = sorted(list(dir_lst[s]['df'].columns))
  
  l = []
  for s in l_seeds:
    l.append(dir_lst[s]['cols'])
  l_common = sorted(list(set.intersection(*map(set, l))))
  print(l_common)
  print(len(l_common))

  for s in l_seeds:
    dir_lst[s]['df'] = dir_lst[s]['df'][l_common]
    # dir_lst[s]['df'].to_csv(dir + 'MODIS/' + f + '__' + s + '.csv')

In [ ]:
df = pd.read_csv(dir + 'MODIS/lst_c__seed_90.csv')
df

In [ ]:
df.columns

*** S2 (CI, NDVI, NDWI)

In [ ]:
df = pd.read_csv(dir + 'S2/cloudfree__seed_30.csv')
print(df)
print(list(df.columns))
print(len(df.columns))

In [ ]:
s2_files = ['cloudfree', 'ci', 'ndvi', 'ndwi']
l_seeds = ['seed_30', 'seed_60']
for f in s2_files:
  print(f)
  dir_s2 = {}
  for s in l_seeds:
    dir_s2[s] = {}
    dir_s2[s]['df'] = pd.read_csv(dir + 'S2/' + f + '__' + s + '.csv')
    dir_s2[s]['df'].drop(columns=['20171008'], inplace=True)
    dir_s2[s]['df'].to_csv(dir + 'S2/' + f + '__' + s + '.csv')

In [ ]:
df = pd.read_csv(dir + 'S2/cloudfree__seed_30.csv')
print(df)
print(list(df.columns))
print(len(df.columns))

In [ ]:
list(df.columns)

In [ ]:
dfs = {}

for s in sources:
  dfs[s] = {}
  path = dir + s
  files = sorted([f for f in os.listdir(path) if 'seed' in f])
  print(files)
  for f in files:
    index, seed = f.split("__seed_")
    if index not in dfs[s].keys():
      dfs[s][index] = {}
    dfs[s][index]['seed_' + seed[:-4]] = pd.read_csv(path + '/' + f)
    dfs[s][index]['seed_' + seed[:-4]].drop(['system:index', 'id', '.geo'], axis=1, inplace=True)
    # if index != 'quality':
    if 'Unnamed: 0' in dfs[s][index]['seed_' + seed[:-4]].columns:
      dfs[s][index]['seed_' + seed[:-4]].drop(['Unnamed: 0'], axis=1, inplace=True)
    dfs[s][index]['seed_' + seed[:-4]] = dfs[s][index]['seed_' + seed[:-4]].T
    if s != 'MODIS':
      dfs[s][index]['seed_' + seed[:-4]]['date'] = dfs[s][index]['seed_' + seed[:-4]].index.to_series().apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
    else:
      dfs[s][index]['seed_' + seed[:-4]]['date'] = dfs[s][index]['seed_' + seed[:-4]].index.to_series().apply(lambda x: pd.to_datetime(str(x), format='%Y_%m_%d'))
    date_column = dfs[s][index]['seed_' + seed[:-4]].pop('date')
    dfs[s][index]['seed_' + seed[:-4]].insert(0, 'date', date_column)
    dfs[s][index]['seed_' + seed[:-4]].reset_index(drop=True, inplace=True)
    dfs[s][index]['seed_' + seed[:-4]].sort_index(inplace=True)

In [ ]:
dfs['S1']['b_asc_vh']['seed_120']

In [ ]:
dfs['S1']['b_asc_vh']['seed_30']

In [ ]:
dfs['S2']['cloudfree']['seed_120']

In [ ]:
list(dfs['S2']['cloudfree']['seed_120']['date'])

In [ ]:
dfs['MODIS']['quality']['seed_90']

In [ ]:
dfs['MODIS']['lst_c']['seed_60']

In [ ]:
dfs['MODIS']['lst_k']['seed_120']

In [ ]:
for s in sources:
  for index in dfs[s].keys():
    print(index)
    seeds = sorted(list(dfs[s][index].keys()))
    print(seeds)
    dfs[s][index]['all'] = dfs[s][index][seeds[0]]
    for i in range(1, len(seeds)):
      dfs[s][index][seeds[i]] = dfs[s][index][seeds[i]].drop(['date'], axis=1)
      dfs[s][index][seeds[i]].columns = np.arange(NUM_POINTS_PER_SEED * i, NUM_POINTS_PER_SEED * (i + 1))
      dfs[s][index]['all'] = dfs[s][index]['all'].join(dfs[s][index][seeds[i]])

In [ ]:
dfs['S1']['b_asc_vh']['all']

In [ ]:
dfs['S2']['ci']['all']

In [ ]:
dfs['S2']['cloudfree']['all']

In [ ]:
dfs['MODIS']['quality']['all']

In [ ]:
dfs['MODIS']['lst_c']['all']

In [ ]:
for index in dfs['S2'].keys():
  date_column = dfs['S2'][index]['all'].pop('date')
  if index != 'cloudfree':
    dfs['S2'][index]['all'] = dfs['S2'][index]['all'].mask(dfs['S2']['cloudfree']['all'] != 1.0)
  dfs['S2'][index]['all'].insert(0, 'date', date_column)

In [ ]:
for index in dfs['MODIS'].keys():
  dfs['MODIS'][index]['all']['date'] = dfs['MODIS'][index]['all']['date'].astype(str)

col_names = list(dfs['MODIS']['quality']['all'].columns)
# dfs['MODIS']['quality']['all_filter'] = pd.DataFrame(columns=col_names)
l_q = list(dfs['MODIS']['quality']['all']['date'])
l_c = list(dfs['MODIS']['lst_c']['all']['date'])
l_q_c = [l_q.index(x) for x in l_c]
# dfs['MODIS']['quality']['all_filter'] = dfs['MODIS']['quality']['all'].iloc[l_q_c]
# dfs['MODIS']['quality']['all_filter'].reset_index(inplace=True, drop=True)
dfs['MODIS']['quality']['all'] = dfs['MODIS']['quality']['all'].iloc[l_q_c]
dfs['MODIS']['quality']['all'].reset_index(inplace=True, drop=True)

for index in dfs['MODIS'].keys():
  date_column = dfs['MODIS'][index]['all'].pop('date')
  if index != 'quality':
    # dfs['MODIS'][index]['all'] = dfs['MODIS'][index]['all'].mask(dfs['MODIS']['quality']['all_filter'] != 1.0)
    dfs['MODIS'][index]['all'] = dfs['MODIS'][index]['all'].mask(dfs['MODIS']['quality']['all'] != 1.0)
  dfs['MODIS'][index]['all'].insert(0, 'date', date_column)

In [ ]:
# dfs['MODIS']['quality']['all_filter']
dfs['MODIS']['quality']['all']

In [ ]:
dfs['MODIS']['lst_c']['all'] 

In [ ]:
dfs['S2']['cloudfree']['all']

In [ ]:
dfs['S2']['ndvi']['all']

In [ ]:
# print(dfs['MODIS']['lst']['all'])

In [ ]:
for s in sources:
  path = dir + s
  for index in dfs[s].keys():
    dfs[s][index]['all'].to_csv(path + '/' + index + '__all.csv')